In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train, X_valid = X_train_full[:50000], X_train_full[50000:]
y_train, y_valid = y_train_full[:50000], y_train_full[50000:]

X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]


11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3,
                        activation='relu', padding='SAME')

model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test)
X_new = X_test[:10] # pretend we have new images
y_pred = model.predict(X_new)

Epoch 1/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.3701 - accuracy: 0.8916 - val_loss: 13.2914 - val_accuracy: 0.9243
Epoch 2/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.1144 - accuracy: 0.9735 - val_loss: 5.4854 - val_accuracy: 0.9804
Epoch 3/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.0842 - accuracy: 0.9801 - val_loss: 11.5697 - val_accuracy: 0.9627
Epoch 4/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.0819 - accuracy: 0.9815 - val_loss: 4.3380 - val_accuracy: 0.9664
Epoch 5/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.0713 - accuracy: 0.9834 - val_loss: 5.1348 - val_accuracy: 0.9760
Epoch 6/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.0629 - accuracy: 0.9851 - val_loss: 5.6945 - val_accuracy: 0.9720
Epoch 7/10
1563/1563 [==============================] - 33s 21ms/step - loss: 0.0552 - accuracy: 0.9873 - val_loss: 9.8602

In [ ]:
y_test[:10], y_pred

(array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=uint8),
 array([[0.00000000e+00, 0.00000000e+00, 1.83404118e-37, 2.01300964e-37,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
         0.00000000e+00, 1.24262536e-33],
        [0.00000000e+00, 3.06492960e-32, 1.00000000e+00, 9.49365329e-34,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 4.79487228e-22,
         1.56750423e-32, 1.19727669e-38],
        [0.00000000e+00, 1.00000000e+00, 4.94332626e-21, 1.32982297e-22,
         1.40294507e-22, 9.34195779e-23, 1.41633463e-22, 2.21428657e-21,
         5.28825433e-17, 1.80453695e-25],
        [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 1.50582681e-33, 8.00945219e-25, 0.00000000e+00,
         1.48639864e-27, 5.32849332e-31],
        [5.29021405e-31, 2.54849338e-20, 1.66769092e-23, 1.32773327e-24,
         1.00000000e+00, 2.12211630e-24, 1.54254907e-17, 3.54037299e-16,
         1.26893053e-15, 1.18325238e-10],
       

In [ ]:
model.save("drive/My Drive/Colab Models/mnist_cnn_model.h5")

In [ ]:
saved_model_dir = "drive/My Drive/Colab Models/mnist_cnn_model.h5"
model = keras.models.load_model(saved_model_dir)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("drive/My Drive/Colab Models/mnist_cnn_tflite_model.tflite", "wb").write(tflite_model)


5659068